<a href="https://colab.research.google.com/github/flaviagaia/SpaCy/blob/main/4_Treinamento_de_modelo_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Named Entity Recognition (NER)**

Reconhecimento de Entidade Nomeada, é um processo de identificação de entidades predefinidas presentes em um texto, como nome da pessoa, organização, local etc.

É um modelo estatístico treinado em um conjunto de dados rotulado e usado para extrair informações de um determinado conjunto de dados.

# Instalar o spaCy

In [ ]:
!pip install -q spacy==2.2.3

     |████████████████████████████████| 10.4MB 4.3MB/s 
     |████████████████████████████████| 2.2MB 34.6MB/s 


# Download do modelo

Usando um modelo pronto da spaCy, que contém idioma, linguagens, vocabulários, vetores treinados, sintaxes e entidades.

In [ ]:
# Português
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 1.3MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp37-none-any.whl size=21186282 sha256=dae41bde23eddbaf1158064e4c9141d719ea04268329091570c754fc10ea7215
  Stored in directory: /tmp/pip-ephem-wheel-cache-v9k4cyqq/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [ ]:
# Importações e carregamento do modelo:

import spacy
import pt_core_news_sm

nlp = pt_core_news_sm.load()

# Exemplo

In [ ]:
texto = nlp('Brasília é a capital do Brasil desde 21/04/1960') # Texto utilizado

# Identificando as etidades + tipo do texto:
for entidade in texto.ents:
  print(entidade.text, entidade.label_)

Brasília LOC
Brasil LOC


In [ ]:
# Destacando visualmente as entidades e seus tipos:

spacy.displacy.render(texto, style='ent', jupyter=True)

In [ ]:
# Visualizando do Analisador de Dependências:

spacy.displacy.render(texto, style='dep',jupyter=True)

In [ ]:
# Definição do tipo de entidades pré existentes no modelo:

print("PER:", spacy.explain('PER'))
print("LOC:", spacy.explain('LOC'))
print("ORG:", spacy.explain('ORG'))
print("MISC:", spacy.explain('MISC'))

PER: Named person or family.
LOC: Non-GPE locations, mountain ranges, bodies of water
ORG: Companies, agencies, institutions, etc.
MISC: Miscellaneous entities, e.g. events, nationalities, products or works of art


# Treinando o modelo spaCy

Como o modelo atual não identifica nem existe o tipo de entidade 
data, irei treinar o modelo para que ele passe a fazer isso

In [ ]:
# Biblioteca que gera números pseudo-aleatórios:

import random

In [ ]:
# Dados para o treinamento: 

TRAIN_DATA = [
    ("Almira é uma ótima pessoa, gosto muito dela", {"entities": [(0, 6, "PER")]}),
    ("João foi para Bahia nas férias", {"entities": [(14, 19, "LOC"), (0, 4, "PER")]}),
    ("Carlos foi visitar João na casa de praia", {"entities": [(0, 6, "PER"), (19, 23, "PER")]}),
    ("No meio do ano irei para São Paulo fazer mais um curso", {"entities": [(25, 34, "LOC")]}),
    ("O sonho dela era ir para Austrália visitar seu irmão", {"entities": [(25, 34, "LOC")]}),
    ("Em 15/07/1988 nasceu essa linda criança", {"entities": [(3, 13, "DATE"), (18, 24, "LOC")]}),
    ("Data de prisão: 10/01/2018", {"entities": [(16, 26, "DATE")]}),
    ("No dia 01/02/2016 foi decretada a sentença", {"entities": [(7, 17, "DATE")]}),
    ("A data da festa foi 07/05/2018", {"entities": [(20, 30, "DATE")]}),
    ("Dia 07/06/2020 choveu pela manhã", {"entities": [(4, 14, "DATE")]}),
    ("Michael Jackson nasceu dia 29/08/1958", {"entities": [(0, 15, "PER"), ( 27, 37, "DATE")]}),
    ("A viagem foi marcada para o dia 21/03/2021", {"entities": [(32, 42, "DATE")]}),
    ("O casamento, que era dia 12/06/2020, foi adiado para o dia 20/07/2020", {"entities": [(25, 35, "DATE"), (59, 68, "DATE")]}),
    ("Dia 21/04/2020 foi feriado", {"entities": [(4, 14, "DATE")]}),
    ("A entrega foi realizada na manhã do dia 18/03/2020 ", {"entities": [(40, 50, "DATE")]}),
    ("Na noite do dia 17/05/2019 aconteceu um eclipse", {"entities": [(16, 26, "DATE")]}),
    ("Fortaleza é a capital do Ceará", {"entities": [(0, 9, "LOC"), (25, 30, "LOC")]})
]

In [ ]:
# Função para o treinamento:

def train_spacy(data, n_iter):
  TRAIN_DATA = data
  """Carrega o modelo, configura o pipeline e treina o reconhecedor da entidade."""
  nlp = pt_core_news_sm.load()  # Carrega um modelo do spaCy existente
  print("Modelo carregado")

  # Criando os componentes internos do pipeline e adicionando-os ao pipeline
  # nlp.create_pipe funciona para built-ins registrados com spaCy
  if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
  # Caso contrário, obtenha-o para que possa adicionar as labels
  else:
    ner = nlp.get_pipe("ner")

  # Adicionando as labels
  for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
      ner.add_label(ent[2])

  # Obtendo os nomes de outros pipes para desativá-los durante o treinamento
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):  # Treina somente NER
    spacy.util.fix_random_seed()
    optimizer = nlp.begin_training()
    
    for itn in range(n_iter):
      random.seed(10)
      random.shuffle(TRAIN_DATA)
      losses = {}
      
      for texts, annotations in TRAIN_DATA:
        nlp.update(
            [texts],  # batch de textos
            [annotations],  # batch de anotações
            drop=0.2,  # dropout - dificulta a memorização de dados
            sgd=optimizer,
            losses=losses,
        )
      print("Losses", losses)

   # test the trained model
    for text, _ in TRAIN_DATA:
      doc = nlp(text)
      print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
      print("Tokens", [(t.text, t.ent_type_) for t in doc])
      print("\n")
    return nlp

In [ ]:
# Variável que armazena o modelo depois do treinamento:

nlp_ = train_spacy(data=TRAIN_DATA, n_iter=100)

Modelo carregado
Losses {'ner': 44.848168294811884}
Losses {'ner': 53.822771509753174}
Losses {'ner': 45.05330151818915}
Losses {'ner': 59.20461495222331}
Losses {'ner': 42.52364630985086}
Losses {'ner': 33.25874292026583}
Losses {'ner': 38.66124423672828}
Losses {'ner': 38.29894106268598}
Losses {'ner': 35.03439654203292}
Losses {'ner': 42.950552726907574}
Losses {'ner': 30.399363830495794}
Losses {'ner': 37.85244673532191}
Losses {'ner': 34.14475805168365}
Losses {'ner': 38.298511604059556}
Losses {'ner': 49.0249132029777}
Losses {'ner': 32.56729737172343}
Losses {'ner': 40.88467343847164}
Losses {'ner': 39.09734799200854}
Losses {'ner': 29.500817759488072}
Losses {'ner': 45.56896432758805}
Losses {'ner': 41.22303263011408}
Losses {'ner': 46.74303617575271}
Losses {'ner': 38.61291791441867}
Losses {'ner': 35.30528266632605}
Losses {'ner': 41.93925517790572}
Losses {'ner': 49.190709596717134}
Losses {'ner': 26.950738787956332}
Losses {'ner': 35.135440551599885}
Losses {'ner': 18.67338

# Testando o modelo depois do treinamento

In [ ]:
text = nlp_("Ana nasceu no dia 18/06/2018 em Brasília")
spacy.displacy.render(text, style='ent', jupyter=True)

In [ ]:
for entidade in text.ents:
  print(entidade.text, entidade.label_)

Ana PER
18/06/2018 DATE
Brasília LOC


**MODELO TREINADO E FUNCIONANDO!!!**